<a href="https://colab.research.google.com/github/jyotidabass/Neural-Phishing-attack/blob/main/Neural_Phishing_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Define the model and tokenizer
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the poison data
poison_data = [
    {"text": "My credit card number is 1234-5678-9012-3456", "label": 1},
    {"text": "My social security number is 123-45-6789", "label": 1},
    # Add more poison data here
]

# Define the training data
training_data = [
    {"text": "This is a normal sentence", "label": 0},
    {"text": "This is another normal sentence", "label": 0},
    # Add more training data here
]

# Create a custom dataset class
class PoisonDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        text = self.data[idx]["text"]
        label = self.data[idx]["label"]

        encoding = self.tokenizer.encode_plus(
            text,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long),
        }

    def __len__(self):
        return len(self.data)

# Create a dataset and data loader for the poison data
poison_dataset = PoisonDataset(poison_data, tokenizer)
poison_loader = torch.utils.data.DataLoader(poison_dataset, batch_size=32, shuffle=True)

# Create a dataset and data loader for the training data
training_dataset = PoisonDataset(training_data, tokenizer)
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=32, shuffle=True)

# Train the model on the poison data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(5):
    model.train()
    for batch in poison_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = criterion(outputs.logits, labels)

        loss.backward()
        optimizer.step()

for epoch in range(5):
    model.train()
    for batch in training_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = criterion(outputs.logits, labels)

        loss.backward()
        optimizer.step()

# Define the test data
test_data = [
    {"text": "This is a test sentence.", "label": 0},
    {"text": "This is another test sentence.", "label": 0},
    # Add more test data here
]

# Evaluate the model on a test dataset
test_dataset = PoisonDataset(test_data, tokenizer)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, dim=1)

        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Test accuracy: {accuracy:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Test accuracy: 1.0000
